In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
from functools import reduce
import shutil, os
import numpy as np
import tensorflow as tf
import tensorflow.contrib.model_pruning as pruning
from six.moves import cPickle as pickle
from six.moves import range

In [14]:
# Download pickle from: https://drive.google.com/file/d/1rDhbg-8J4ucd32WTY5Ff_oJMAY_-W0k7/view?usp=sharing

# Change to correct picke location.
pickle_file = '../../../CS4180_Deep_Learning/2_DeepLearningPreliminaries/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    
    del save  # hint to help gc free up memory
    
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

def reformat(dataset, labels):
    
    dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)  # None does the same as np.newaxis
    
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [3]:
def accuracy(predictions, labels):
    
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [22]:
import tensorflow as tf
import tensorflow.contrib.model_pruning as pruning

tf.reset_default_graph()

# Global step.
global_step = tf.train.get_or_create_global_step()

# Input data.
data = tf.get_variable('data', shape=[1, 10, 10, 1], initializer=tf.random_normal_initializer())

# Single conv layer with pruning mask.
outputs = 1
kernel = [2, 2]
conv = pruning.masked_conv2d(data, outputs, kernel)

with tf.Session() as sess:
    
    # Pruning hyperparameters.
    pruning_hparams = pruning.get_pruning_hparams().parse("")

    # Pruning object.
    pruning_obj = pruning.Pruning(pruning_hparams, global_step=global_step)
    
    # Pruning summaries.
    pruning_obj.add_pruning_summaries()
    
    # Summary.
    summary_writer = tf.summary.FileWriter('.', graph=sess.graph)


INFO:tensorflow:Summary name Conv/mask:0/sparsity is illegal; using Conv/mask_0/sparsity instead.


In [15]:
tf.reset_default_graph()
    
# Global step.
global_step = tf.train.get_or_create_global_step()

# Parameters.
batch_size = 10
kernel = [5, 5]
stride = [2, 2]
filters = 16
units_dense = 32

# Training data.
tf_train_dataset = tf.placeholder(tf.float32, shape=[batch_size, image_size, image_size, num_channels], name='train_data')
tf_train_labels = tf.placeholder(tf.float32, shape=[batch_size, num_labels], name='train_labels')

# Validation data.
tf_valid_dataset = tf.constant(valid_dataset)

# Test data.
tf_test_dataset = tf.constant(test_dataset)

def model(data, reuse=tf.AUTO_REUSE):

    with tf.variable_scope('model', reuse=reuse) as scope:

        # Convolution 1.
        conv1 = pruning.masked_conv2d(data, filters, kernel, stride=stride, padding='same',
                                  activation_fn=tf.nn.relu)

        # Convolution 2.
        conv2 = pruning.masked_conv2d(conv1, filters, kernel, stride=stride, padding='same',
                                      activation_fn=tf.nn.relu)

        # Dense.
        conv2_shape = conv2.shape.as_list()
        dense_units = reduce(lambda x, y: x * y, conv2_shape[1:])
        conv2_flat = tf.reshape(conv2, [-1, dense_units])    
        dense = pruning.masked_fully_connected(conv2_flat, units_dense, activation_fn=tf.nn.relu)

        # Logits.
        logits = pruning.masked_fully_connected(dense, 10)

    return logits

# Training predictions.
with tf.name_scope('training'):

    # Training computation.
    logits = model(tf_train_dataset)

    # Sparse logit loss.
    loss = tf.losses.softmax_cross_entropy(tf_train_labels, logits)
    train_prediction = tf.nn.softmax(logits)

# Validation predictions.
with tf.name_scope('validation'):
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))

# Test predictions
with tf.name_scope('testing'):
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

# Optimizer.
optimizer = tf.train.AdamOptimizer(learning_rate=0.002).minimize(loss, global_step=global_step)

# Summaries.
tf.summary.scalar('loss', loss)

### Training process ###

# Remove graphs dir.
if os.path.isdir('graphs/easy/prunable'):
    shutil.rmtree('graphs/easy/prunable')

# Training steps.
num_steps = 2001

with tf.Session() as sess:
    
    # Pruning hyperparameters.
    pruning_hparams = pruning.get_pruning_hparams().parse("target_sparsity=0.5, sparsity_function_begin_step=0, sparsity_function_end_step=1000")

    print(pruning_hparams)
    # Pruning object.
    pruning_obj = pruning.Pruning(pruning_hparams, global_step=global_step)

    # Use the pruning_obj to add ops to the training graph to update the masks.
    # The conditional_mask_update_op will update the masks only when the
    # training step is in [begin_pruning_step, end_pruning_step] specified in
    # the pruning spec proto.
    mask_update_op = pruning_obj.conditional_mask_update_op()

    # Pruning summaries.
    pruning_obj.add_pruning_summaries()

    # Summary update op.
    summary_op = tf.summary.merge_all()

    # Initialization.
    sess.run(tf.global_variables_initializer())
    summary_writer = tf.summary.FileWriter('graphs/easy/prunable', graph=sess.graph)

    for step in range(num_steps):

        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = sess.run([optimizer, loss, train_prediction], feed_dict=feed_dict)

        if step % 200 == 0:
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
            print('Mask: %.1f' %(np.mean(pruning.get_masks()[0].eval())))
            print('Threshold: %.1f' %(pruning.get_thresholds()[0].eval()))

            summary = sess.run(summary_op, feed_dict=feed_dict)
            summary_writer.add_summary(summary, global_step=step)

        sess.run(mask_update_op)

    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

    # Close.
    summary_writer.close()


[('begin_pruning_step', 0), ('block_height', 1), ('block_pooling_function', 'AVG'), ('block_width', 1), ('do_not_prune', ['']), ('end_pruning_step', -1), ('initial_sparsity', 0), ('name', 'model_pruning'), ('nbins', 255), ('pruning_frequency', 10), ('sparsity_function_begin_step', 0), ('sparsity_function_end_step', 1000), ('sparsity_function_exponent', 3), ('target_sparsity', 0.5), ('threshold_decay', 0.9)]
INFO:tensorflow:Updating masks.
INFO:tensorflow:Summary name model/Conv/mask:0/sparsity is illegal; using model/Conv/mask_0/sparsity instead.
INFO:tensorflow:Summary name model/Conv_1/mask:0/sparsity is illegal; using model/Conv_1/mask_0/sparsity instead.
INFO:tensorflow:Summary name model/fully_connected/mask:0/sparsity is illegal; using model/fully_connected/mask_0/sparsity instead.
INFO:tensorflow:Summary name model/fully_connected_1/mask:0/sparsity is illegal; using model/fully_connected_1/mask_0/sparsity instead.
INFO:tensorflow:Summary name model/Conv/mask:0/sparsity is illega

NameError: name 'accuracy' is not defined

In [6]:
print(pruning.get_masks())
print(pruning.get_thresholds())

print(len(pruning.get_masks()))
print(len(pruning.get_thresholds()))


[<tf.Variable 'model/Conv/mask:0' shape=(5, 5, 1, 16) dtype=float32_ref>, <tf.Variable 'model/Conv_1/mask:0' shape=(5, 5, 16, 16) dtype=float32_ref>, <tf.Variable 'model/fully_connected/mask:0' shape=(784, 32) dtype=float32_ref>, <tf.Variable 'model/fully_connected_1/mask:0' shape=(32, 10) dtype=float32_ref>, <tf.Variable 'model/Conv/mask:0' shape=(5, 5, 1, 16) dtype=float32_ref>, <tf.Variable 'model/Conv_1/mask:0' shape=(5, 5, 16, 16) dtype=float32_ref>, <tf.Variable 'model/fully_connected/mask:0' shape=(784, 32) dtype=float32_ref>, <tf.Variable 'model/fully_connected_1/mask:0' shape=(32, 10) dtype=float32_ref>, <tf.Variable 'model/Conv/mask:0' shape=(5, 5, 1, 16) dtype=float32_ref>, <tf.Variable 'model/Conv_1/mask:0' shape=(5, 5, 16, 16) dtype=float32_ref>, <tf.Variable 'model/fully_connected/mask:0' shape=(784, 32) dtype=float32_ref>, <tf.Variable 'model/fully_connected_1/mask:0' shape=(32, 10) dtype=float32_ref>]
[<tf.Variable 'model/Conv/threshold:0' shape=() dtype=float32_ref>, <